# Harmonization of the Rhythmized Melody

Simply specify the parameters in the second code-cell and then let all the code-cells run (at least to the optional hyperparameter-search part) :)

## Importing Libraries

In [1]:
#### IMPORTS ####

import partitura as pt
import numpy as np
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'
from partitura.utils.synth import synthesize

import IPython.display as ipd
from helpers_EA import (partFromFourPartProgression, randomword)

## Parameters

Here, you can change the parameters for the music-harmonization-pipeline:

1. `in_midi` specifies the `performance`-object of a given midi-file which contains a melody. Alternatively every file consisting of a melody, which is supported by the `Partitura`-Package can be specified here, as long it can be converted into a performance. In the simplest case, you can change the midi-file-path here to a custom one, since the `pt.load_performance_midi()` function automatically loads a midi-file from the file-system into a `performance`-object.
2. `in_scale` must be a numpy-array, which specifies the musical scale of the musical piece starting with 0 for the root-note. The numbers correspond to semitones, such that for a major-scale the input would be `np.array([0,2,4,5,7,9,11])`, because the semitones in a major-scale are between the 3rd and the 4th step as well as the 7th and the 8th step (everything else is a whole step). Note, that `in_scale` only specifies one octave starting from 0, and it is agnostic towards the actual pitch of the root-note. Only the semitone-steps are important - the actual key of the scale is specified in `in_key`.
3. `in_key` represents the key in which the piece should be in. The values correspond to MIDI-pitches, where 48 represents a C3 (see: https://www.inspiredacoustics.com/en/MIDI_note_numbers_and_center_frequencies).
4. If there are missing computational-capacities, there is the option to specify how many midi-notes are used for harmonization, which can be specified in `first_n_notes_to_be_used`. If this value is set to e.g. 10, then only the first 10 notes of the midi-input will be used for harmonization.
5. The pipeline exports the harmonized MIDI-piece into a MIDI-file, as well as a wav-file by using additive synthesis. `file_name` specifies the filename for the output-files. These will be stored as `filename`.wav and `filename`.mid.

The next parameters are more technical ones specifying the details of the underlying evolutionary-algorithm.
1. `population_size` sets the number of population in the evolutionary algorithm. Basically every generated chord-progression is treated as a possible candidate for the optimal one, where a fitness-function evaluates which of them is the best. `population_size` basically sets the number of candidate solutions. In the beginning, there will be n chord-progressions randomly initialized, where n is set to the `population_size`.
2. After each epoch, some chords of the chord progressions are changed to get a better score for the fitness-function. This behavior is specified by the `replacement_ratio`. This parameter defines how many of the n possible candidate-solutions will be replaced in percent. When replacing a solution (chord-progression), there are two things involved. At first, two existing candidate-solutions can be merged together to form a new solution.
3. Therefore, the `combination_ratio` defines how two solutions will be merged together and how many of them will influence the newly generated solution.
4. After generating a new solution by merging two possible candidate-solutions together, there are also some mutations involved. This basically means, that some of the chords will be replaced in a random way to try out new possibilities which could fit the fitness-score better. This behavior is determined by `mutation_ratio`.

Then some general training-parameters follow:
1. `epochs` defines how many times in a loop different chord progressions are tried out to find the optimal one regarding the fitness-function.
2. `new_population` specifies if we want to generate a new population each epoch, which should be set to True, if you want to see some progress in training.
3. `max_epochs_without_improvement` is similar to the early-stopping parameter, but without a validation set. This basically means, that the training-process aborts if there is no improvement seen in the fitness-score-values for n epochs, where n is defined by `max_epochs_without_improvement`.

The last parameter is set in `chord_types`, which specifies the different chord-types which are allowed when generating a solution. These chord-types are defined in triples, because one chord consists of 4 notes in our setting, and one of them is the melody. In this way, you could restrict the chords used only to ones, which do not double the interval of the third within a chord.

In [2]:
#### PARAMETERS TO SPECIFY ####

in_midi = pt.load_performance_midi("rhythmized.mid")

# MUST BE A NUMPY ARRAY!
# major: [0,2,4,5,7,9,11], minor: [0,2,3,5,7,8,10], chromatic: [0,1,2,3,4,5,6,7,8,9,10]
in_scale = np.array([0,2,3,5,7,8,10])

# 48 = C
in_key = 48                             

# can restrict the size of the output
# use len(in_midi.note_array()) if the entire melody should be processed
first_n_notes_to_be_used = len(in_midi.note_array()) 

# will produce a <file_name>.mid and <file_name>.wav file
file_name = "FinalPiece"              

# will be optimized during (an optional) hyperparameter search
# when using the normal generation-task, then the parameters have to be set accordingly
population_size = 100
replacement_ratio = 0.2
combination_ratio = 0.2
mutation_ratio = 0.2

#### TRAINING PARAMETERS ####

epochs = 2000
new_population = True
max_epochs_without_improvement = 100    # early stopping parameter

#### TRAINING PARAMETERS ####

#### COMMENT OUT CHORDS WHICH SHOULD NOT BE USED ####

chord_types = {
    # position 1: soprano in C
    "a":(10,3,4),
    "b":(3,3,4),
    "c":(12,6,6),
    "d":(5,6,6),
    "e":(6,5,4), # 6
    "f":(14,5,4), # 6
    "g":(3,4,8), # 6
    "h":(10,4,8), # 6
    "i":(3,8,4), # 6 (5 doubled)
    "j":(10,8,4), # 6 (5 doubled)
    
    # position 2: soprano in G
    "k":(8,3,3),
    "l":(1,3,3),
    "m":(5,6,3), #(5 doubled)
    "n":(12,6,3), #(5 doubled)
    "o":(3,6,5),
    "p":(10,6,5),
    "q":(3,3,8), # (5 doubled)
    "r":(10,3,8), # (5 doubled)
    "s":(6,8,5), # 6
    "t":(14,8,5), # 6 
    "u":(3,4,5), # 6 (5 doubled)
    "v":(10,4,5), # 6 (5 doubled)
    "w":(6,5,8), # 6 (5 doubled)
    "x":(14,5,8), # 6 (5 doubled)
    
    # position 3: soprano in E
    "y":(1,5,6), 
    "z":(8,5,6),
    "ö":(5,4,3),
    "ü":(12,4,3),
    "ä":(5,8,6), # (5 doubled)
    ".":(12,8,6), # (5 doubled) 
}

#### PARAMETERS TO SPECIFY ####

C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=1 note=70 velocity=0 time=1008
  warnings.warn(f"ignoring MIDI message {msg}")


## Evolutionary Algorithm - Pipeline

### Pitch Correction

After setting the parameters for the evolutionary algorithm, at first the pitch array will be corrected. There might be notes which part of the melody but do not correspond to the given scale determined by `in_scale` and `in_key`. These notes will be corrected by pitching them up one semitone to fit into the scale respectively.

In [3]:
# getting pitches of note array
in_note_array = in_midi[0].note_array()
in_note_array_pitch = in_note_array["pitch"]

In [4]:
# adds +1 to pitches which are not in scale
def correct_pitches(note_array_pitch, in_scale, in_key):
    lowest_offset = in_key % 12
    whole_scale = np.concatenate([in_scale + lowest_offset + 12 * octave  
                                               for octave in np.arange(10)])
    corrected_notes = np.copy(note_array_pitch)
    for idx, note in enumerate(note_array_pitch):
        if note not in set(whole_scale):
            corrected_notes[idx] += 1
    
    return corrected_notes

In [5]:
in_note_array_pitch_corrected = correct_pitches(in_note_array_pitch, in_scale, in_key)

### Implementation of the Evoluationary Algorithm Pipeline

The Evolutionary Algorithm Pipeline is implemented in the function `generate_midi()` and reuses code from the Music Informatics lecture. This code is adapted to change with all the parameters provided in the parameter-section. Therefore, the three helper-classes `FourPartChord`, `FourPartProgression` and `FourPartOptimizer` were modified.

Basically the main-input for the melody is an array of the corrected pitches (see above). With that pitch-information a harmony is created, which is not rhythmized directly as part of the Evoluationary Algorithm, because this task was already accomplished by the Rhythmizer of the piece. The direct output of this algorithm is a `partitura.score`-object of the package `Partitura` which is used for Music Information Retrieval (MIR). This score-object contains a note-array with the chord-information for the provided pitches (without the rhythm).

To use the rhythmic information of the original note array provided by `in_midi`, at first the `partitura.score`-object must be converted into a `partitura.performance`-object, since `in_midi` is also of the same datatype. This cannot be done directly within `Partitura`, which is the reason why we have to output a temporary midi-file and load it again as a `partitura.performance`-object by using `pt.save_score_midi()` and `pt.load_performance_midi()`. 

After having loaded the output of the harmonizer as a `partitura.performance`-object, we extracted the note-array-information of it, and then we overwrote the duration and onset-information of the notes by using the values provided by original melody of `in_midi`. This cannot be done 1:1, because now there are four notes (representing the harmony) corresponding to one melody note in the original melody, why an additional for-loop has to be introduced. In the end, every MIDI-Note was set to channel 1 to only have one MIDI-Channel in the output for convenience when opening it inside a DAW.

The last step consists of storing the modified note-array into a `partitura.performance`-object and then saving that as a midi and wav-file with the filename specified by the parameter `file_name`. Additionally, `generate_midi()` returns the final `partitura.performance`-object with the harmonized note-array as well as the final fitness-score.

Also, if you do not want to output the intermediate results of the fitness-function, you can set `show_output` to False.

In [6]:
def generate_midi(output_file_name, 
                  chord_types, 
                  note_pitches_melody, 
                  original_note_array, 
                  in_scale, in_key, 
                  max_epochs_without_improvement=10, 
                  epochs=200, 
                  population_size=100, 
                  replacement_ratio=0.2, 
                  mutation_ratio=0.2, 
                  combination_ratio=0.2, 
                  new_population=True, 
                  show_output=True):    # if True, the intermediate results of the fitness-function during the epochs will be printed to the console
    
    #### DEFINITION OF HELPER-CLASSES ####
    
    class FourPartChord:
        """
            the Chord class representing a chord made of 4 notes of a scale.
            For diatonic scales the intervals between notes are thirds.
        
            Parameters
            ----------
            chord_type: str
                the type of the chord
            offset : int
                the MIDI pitch of scale degree 0
            scale : np.array
                the scale from which the chord is built

        """
        def __init__(self, 
                     chord_type = "a",
                     offset = in_key,
                     scale = in_scale,
                    ):
            self.chord_type = chord_type
            self.offset = offset
            self.scale = scale
            lowest_offset = offset % 12
            self.whole_scale = np.concatenate([self.scale + lowest_offset + 12 * octave  
                                               for octave in np.arange(10)])
            self.intervals = chord_types[self.chord_type]
            self.soprano = None
            self.alto = None
            self.tenor = None
            self.bass = None
            self.pitch_computed = False
            self.id = randomword(4)
            
        def compute_pitch(self,
                          given_pitch, # midi pitch of given melody
                          given_voice): # voice (0-3) of given melody
            if not self.pitch_computed:
                given_note_idx = np.where(self.whole_scale==given_pitch)[0]
                
                if len(self.whole_scale) == 0:
                    raise ValueError("given note needs to be in the scale")
                
                if given_voice == 0: 
                    self.soprano = self.whole_scale[given_note_idx]
                    self.alto = self.whole_scale[given_note_idx - (self.intervals[2] - 1) ]
                    self.tenor = self.whole_scale[given_note_idx - (self.intervals[2] - 1) - (self.intervals[1] - 1) ]
                    self.bass = self.whole_scale[given_note_idx - (self.intervals[2] - 1) - (self.intervals[1] - 1) - (self.intervals[0] - 1) ]
                elif given_voice == 1: 
                    self.soprano = self.whole_scale[given_note_idx + (self.intervals[2] - 1) ]
                    self.alto = self.whole_scale[given_note_idx ]
                    self.tenor = self.whole_scale[given_note_idx - (self.intervals[1] - 1) ]
                    self.bass = self.whole_scale[given_note_idx - (self.intervals[1] - 1) - (self.intervals[0] - 1)  ]
                elif given_voice == 2: 
                    self.soprano = self.whole_scale[given_note_idx + (self.intervals[2] - 1) + (self.intervals[1] - 1)]
                    self.alto = self.whole_scale[given_note_idx + (self.intervals[1] - 1) ]
                    self.tenor = self.whole_scale[given_note_idx]
                    self.bass = self.whole_scale[given_note_idx - (self.intervals[0] - 1) ] 
                elif given_voice == 3: 
                    self.soprano = self.whole_scale[given_note_idx + (self.intervals[2] - 1) + (self.intervals[1] - 1) + (self.intervals[0] - 1)]
                    self.alto = self.whole_scale[given_note_idx + (self.intervals[1] - 1) + (self.intervals[0] - 1) ]
                    self.tenor = self.whole_scale[given_note_idx + (self.intervals[0] - 1)]
                    self.bass = self.whole_scale[given_note_idx]  
    
                self.pitch_computed = True
                
    class FourPartProgression:
        """
        the Progression class representing a sequence of chords
        """
        def __init__(self,
                     c_type_sample = None,
                     number_of_chords = 8,
                     offset = in_key,
                     scale = in_scale):
            
            self.c_types = list(chord_types.keys())
            if c_type_sample is not None:
                self.c_type_sample = np.random.choice(self.c_types, number_of_chords)
            else:
                self.c_type_sample = np.random.choice(self.c_types, number_of_chords)
            self.chords = [FourPartChord(self.c_type_sample[c], offset = offset, scale = scale) for c in range(number_of_chords)]
            self.number_of_chords = number_of_chords
            self.id = randomword(10)
        
        def copy(self):
            return FourPartProgression(number_of_chords = self.number_of_chords,
                                       c_type_sample = self.c_type_sample)
        
        def set_voice(self, melody, voice):
            for cidx, melody_pitch in enumerate(melody):
                self.chords[cidx].compute_pitch(melody_pitch, voice)
    
        def point_mutate(self, idx = None, c_type_sample = None):   
            if idx is None:
                idx = np.random.randint(0,self.number_of_chords)
            if c_type_sample is None:
                c_type_sample = np.random.choice(self.c_types, 1)
            self.c_type_sample[idx] = c_type_sample[0]
            self.chords[idx] = FourPartChord(c_type_sample[0])
            
        def join(self, 
                 another,
                 idx = None):
            """
            create two new progressions by joining two existing ones
            where the split of chords is defined by an index array
            """
            if another.number_of_chords != self.number_of_chords:
                raise ValueError("progressions must have the same number of chords")
    
            if idx is None:
                idx = np.unique(np.random.randint(0,self.number_of_chords,self.number_of_chords//2))
    
            new_progression = FourPartProgression(number_of_chords = self.number_of_chords)
            new_another_progression = FourPartProgression(number_of_chords = self.number_of_chords)
            for k in range(self.number_of_chords):
                if k in idx:
                    new_progression.chords[k] = self.chords[k]
                    new_another_progression.chords[k] = another.chords[k]
                else:
                    new_progression.chords[k] = another.chords[k]
                    new_another_progression.chords[k] = self.chords[k]
            return new_progression, new_another_progression
    
    class FourPartOptimizer:
        def __init__(self):
            self.population = None   
    
        def modify(self, population, number_to_mutate, number_to_recombine):
            # point mutation of chord
            subpop4 = np.random.choice(population, number_to_mutate)
            for element in subpop4:
                cidx = np.random.randint(len(element.chords))
                new_element = element.copy()
                new_element.point_mutate(cidx)
                population.append(new_element)
                
            # recombination of sequence
            subpop1 = np.random.choice(population, number_to_recombine)
            subpop2 = np.random.choice(population, number_to_recombine)
            for element0, element1 in zip(subpop1, subpop2):
                elnew1, elnew2 = element0.join(element1)
                population.append(elnew1)
                population.append(elnew2)
            
            return population
        
        def fitness(self, progression, melody, voice):
            progression.set_voice(melody, voice)
            index_range = (0, progression.number_of_chords)
            fit = 0.0
            for c0, c1 in zip(progression.chords[index_range[0]:index_range[1]], 
                              progression.chords[index_range[0]+1:index_range[1]+1]):
                melodic_lines = np.array([c1.soprano - c0.soprano, c1.alto - c0.alto, c1.tenor - c0.tenor])
                melodic_lines_bass = np.array([c1.soprano - c0.soprano, c1.alto - c0.alto, c1.tenor - c0.tenor, c1.bass - c0.bass])
                fit += np.sum(np.abs(melodic_lines)) # penalize large leaps
                if (melodic_lines_bass > 0).sum() >= 3:
                    fit += 5 # similar motion
                elif (melodic_lines_bass < 0).sum() >= 3:
                    fit += 5 # similar motion
    
                intervals0 = np.array([c0.soprano - c0.alto, c0.alto - c0.tenor, c0.soprano - c0.tenor])
                intervals1 = np.array([c1.soprano - c1.alto, c1.alto - c1.tenor, c1.soprano - c1.tenor])
                for i0, i1 in zip(intervals0, intervals1):
                    if i0 == i1: 
                        if np.abs(i0) == 7:
                            fit += 10 # parallel fifths
                        elif np.abs(i0) == 12:
                            fit += 10 # parallel octave
                
            # add a small random number for hashing
            fit += np.random.rand(1)[0]
            return float(fit) 
        
        def select(self, population, melody, voice, number_to_keep):
            pop = {ele.id:ele for ele in population}
            fitness_dict = {self.fitness(ele, melody, voice):ele.id for ele in population}
            sorted_fitness = list(fitness_dict.keys())
            sorted_fitness.sort()
            new_pop = [pop[fitness_dict[k]] for k in sorted_fitness[:number_to_keep]]
            best_prog = pop[fitness_dict[sorted_fitness[0]]]
            return new_pop, sorted_fitness, best_prog
    
        def run(self,
                epochs = epochs,
                population_size = population_size,
                replacement_ratio = replacement_ratio,
                mutation_ratio = mutation_ratio,
                combination_ratio = combination_ratio,
                new_population = new_population,
                melody = note_pitches_melody,   # C = 48, D = 50, E = 52, F = 53, G = 55, A = 57, H = 59
                voice = 1,
                number_of_chords = len(note_pitches_melody),
                save_part = 40):
            
            cur_fitness = np.inf
            score_fitness = np.inf
            count = 0
            part = None
            number_to_keep = int((1 - replacement_ratio) * population_size)
            number_to_sample = int((replacement_ratio) * population_size)
            number_to_mutate = int(mutation_ratio  * population_size)
            number_to_recombine = int(combination_ratio  * population_size)
    
            if new_population:
                population = [FourPartProgression(number_of_chords = number_of_chords) for po in range(population_size)]    
            else:
                population = self.population
            
            for epoch in range(epochs): 
                population = self.modify(population, number_to_mutate, number_to_recombine) 
                population, sorted_fitness, best_prog = self.select(population, melody, voice, number_to_keep)
                
                if show_output:
                    print(f"Epoch {epoch} best fitness: {sorted_fitness[0]:.4f}")
                    
                if epoch % save_part == 0:
                    part = partFromFourPartProgression(best_prog, 
                                    part = part,
                                    quarter_duration = 1,
                                    time_offset = (epoch // save_part) *8)
                
                population += [FourPartProgression(number_of_chords = number_of_chords) for po in range(number_to_sample)]
                
                # checks if the fitness improves, else, it breaks the loop
                score_fitness = sorted_fitness[0]
                if score_fitness < cur_fitness:
                    cur_fitness = score_fitness
                    count = 0
                else:
                    count += 1
                    
                if count > max_epochs_without_improvement:
                    break
                
            self.population = population
            
            pt.score.add_measures(part)
            
            return self.population, part, score_fitness
    
    #### DEFINITION OF HELPER-CLASSES ####
    
    #### HARMONIZATION PROCESS ####
    
    print("START OF HARMONIZATION PROCESS... \n")
    
    # prints some information about the given hyperparameters
    print(f"FirstNotes: {first_n_notes_to_be_used}\n"
          f"PopSize: {population_size}\n"
          f"ReplRatio: {replacement_ratio}\n"
          f"MutRatio: {mutation_ratio}\n"
          f"CombRatio: {combination_ratio}\n\n")
    
    exp = FourPartOptimizer()
    population, part, score_fitness = exp.run()
    
    print(f"Fitness: {score_fitness}")
    print("\nEND OF HARMONIZATION PROCESS")
    
    #### HARMONIZATION PROCESS ####
    
    ### converts the score-file into a performance-file for further processing ###
    pt.save_score_midi(part, "temp.mid")
    midi = pt.load_performance_midi("temp.mid")
    part = midi
    
    ### takes the first entries of the note-array ###
    no_of_chords = 4
    idx_output = len(note_pitches_melody) * no_of_chords
    output_note_array = part.note_array()[:idx_output]
    
    ### corrects the generated note-array by onset and duration-values and generate output-midi ###
    count_iteration = 0
    for idx in range(0, len(original_note_array)*no_of_chords, no_of_chords):
        for note_offset in range(no_of_chords):
            output_note_array[idx+note_offset]["onset_sec"] = original_note_array[count_iteration]["onset_sec"]
            output_note_array[idx+note_offset]["duration_sec"] = original_note_array[count_iteration]["duration_sec"]
            output_note_array[idx+note_offset]["onset_tick"] = original_note_array[count_iteration]["onset_tick"]
            output_note_array[idx+note_offset]["duration_tick"] = original_note_array[count_iteration]["duration_tick"]
            output_note_array[idx+note_offset]["velocity"] = original_note_array[count_iteration]["velocity"]
            
            # setting the channel to 1, to have the chords only in one midi-channel
            output_note_array[idx+note_offset]["channel"] = 1
            
        count_iteration += 1
    
    ### generates performance out of note-array  ###
    pp = pt.performance.PerformedPart.from_note_array(output_note_array)
    pt.save_wav(pp, output_file_name + '.wav')
    pt.save_performance_midi(pp, output_file_name + '.mid')
    
    return pp, score_fitness

### Generation of the MIDI-Piece

This code-cell calls the evolutionary-algorithm-pipeline created in `generate_midi()`. The fitness-score value is ignored, because it is printed to the console.

In [7]:
piece, _ = generate_midi(
    file_name, 
    chord_types, 
    in_note_array_pitch_corrected[:first_n_notes_to_be_used], 
    in_note_array[:first_n_notes_to_be_used], 
    in_scale, 
    in_key, 
    max_epochs_without_improvement, 
    epochs, population_size, 
    replacement_ratio, 
    mutation_ratio, 
    combination_ratio, 
    new_population
)

START OF HARMONIZATION PROCESS... 

FirstNotes: 20
PopSize: 100
ReplRatio: 0.2
MutRatio: 0.2
CombRatio: 0.2

Epoch 0 best fitness: 708.8237
Epoch 1 best fitness: 700.7794
Epoch 2 best fitness: 700.3207
Epoch 3 best fitness: 696.0460
Epoch 4 best fitness: 696.1383
Epoch 5 best fitness: 686.2173
Epoch 6 best fitness: 668.0778
Epoch 7 best fitness: 656.5289
Epoch 8 best fitness: 656.3972
Epoch 9 best fitness: 656.1646
Epoch 10 best fitness: 656.4996
Epoch 11 best fitness: 656.6931
Epoch 12 best fitness: 656.4554
Epoch 13 best fitness: 625.2807
Epoch 14 best fitness: 625.3794
Epoch 15 best fitness: 625.0449
Epoch 16 best fitness: 621.8982
Epoch 17 best fitness: 621.7081
Epoch 18 best fitness: 621.9459
Epoch 19 best fitness: 621.7184
Epoch 20 best fitness: 621.4626
Epoch 21 best fitness: 621.5973
Epoch 22 best fitness: 614.8815
Epoch 23 best fitness: 614.7419
Epoch 24 best fitness: 614.0221
Epoch 25 best fitness: 614.7344
Epoch 26 best fitness: 613.4339
Epoch 27 best fitness: 613.0888
Epoch

C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=4 note=65 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=1 note=98 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=2 note=94 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=2 note=84 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off chann

### Playback of the MIDI-Piece

For testing the result, the following code-cell provides an audio-player for listening to the harmonized melody synthesized by an additive synthesizer. The code is adapted from the Music Informatics course.

In [8]:
SAMPLE_RATE = 44100

audio = synthesize(piece, samplerate=SAMPLE_RATE)
audio /= 4 # a bit quieter
ipd.display(ipd.Audio(data=audio, rate=SAMPLE_RATE, normalize=False))

## Hyperparameter-Search (optional)

This part is optional:

When performing the hyperparameter-search, essentially the melody in `in_midi`, the `in_scale`, the `in_key` as well as the `first_n_notes_to_be_used`, the `file_name` and the `chord_types` are fixed to the values specified in the parameter-section of this file.

The hyperparameter-search then tries to find the best fitness-score by trying out different combinations of the `population_size`, the `replacement_ratio`, the `mutation_ratio` and the `combination_ratio`, which can be specified below.

### Defining the parameter space

In [9]:
# importing the library for creating an iterator which iterates over the parameter space
import itertools as it

In [10]:
param_space = {
                   "population_size": [25, 50, 100, 200, 400, 600, 1000],   # default: 100
                   "replacement_ratio": [0.1, 0.2, 0.4, 0.6, 0.8, 0.9],     # default: 0.2
                   "mutation_ratio": [0.1, 0.2, 0.4, 0.6, 0.8, 0.9],        # default: 0.2
                   "combination_ratio": [0.1, 0.2, 0.4, 0.6, 0.8, 0.9]      # default: 0.2
               }

# creates list containing all the hyperparameter configurations as tuples
combinations_param = it.product(*(param_space[item] for item in param_space))
combination_list = list(combinations_param)

The only thing, one needs to be aware of when doing hyperparameter search is the combinatorial explosion. Since every combination of the values specified in the parameter-space needs to be explored, the number of combinations is very high:

In [11]:
len(combination_list)

1512

To just test the functionality of the hyperparameter-search, we provided a smaller parameter-space which can be used when setting `use_test_configuration` to True. Otherwise, the parameter space specified above will be used.

In [12]:
use_test_configuration = False

In [13]:
param_space_test = {
                    "population_size": [25, 50],         # default: 100
                    "replacement_ratio": [0.1, 0.2],     # default: 0.2
                    "mutation_ratio": [0.1, 0.2],        # default: 0.2
                    "combination_ratio": [0.1]           # default: 0.2
                    }

# creates list containing all the hyperparameter configurations as tuples
combinations_param_test = it.product(*(param_space_test[item] for item in param_space_test))
combination_list_test = list(combinations_param_test)

This test-hyperparameter-search only consists of 8 possible hyperparameter-configurations.

In [14]:
len(combination_list_test)

8

### Implementation of the Hyperparameter-Search

The next code-cell executes the hyperparameter-search. 

During this procedure every harmonized melody with a corresponding hyperparameter-combination is stored in a separate file with the name `HyperParamSearch_FirstNotes_{first_n_notes_to_be_used}_PopSize_{pop_size}_ReplRatio_{repl_ratio}_MutRatio_{mut_ratio}_CombRatio_{comb_ratio}`. The placeholders specified in brackets are replaced by the hyperparameters, additionally the fixed parameter `first_n_notes_to_be_used` is also part of this name. 

Generally, these files would be stored in the same directory as this jupyter-notebook-file, but to not store too many different files in one directory, another folder is created inside the current working directory with the name `hyperparam` by the method `mkdir` which is part of the python `os` library. Sometimes errors can occur, if this folder already exists.

Additionally, in this directory also a log file `Log.txt` is created which stores the best-score for every hyperparameter-configuration. When the hyperparameter-search is finished, the best performing hyperparameter-configuration and its score are stored separately inside the log-file in the end. The scores will be as well printed to the console.

In [15]:
# sets the list containing all the hyperparameter configurations according to the parameter "use_test_configuration"
if use_test_configuration:
    comb_list = combination_list_test
else:
    comb_list = combination_list

os.mkdir("hyperparam")
log_file = "./hyperparam/Log.txt"
log_file_pointer = open(log_file, "w")

# the best score is stored here additionally to the corresponding hyperparameter-configuration
# which is part of the file name
best_score = np.inf     # the smaller, the better
best_file_name = " "

# iterates through every hyperparameter-combination
for item in comb_list:
    
    # extraction of the parameters in the current hyperparameter configuration
    pop_size = item[0]
    repl_ratio = item[1]
    mut_ratio = item[2]
    comb_ratio = item[3]
    
    file_name = f"./hyperparam/HyperParamSearch_FirstNotes_{first_n_notes_to_be_used}_PopSize_{pop_size}_ReplRatio_{repl_ratio}_MutRatio_{mut_ratio}_CombRatio_{comb_ratio}"
    
    # generate_midi() is called
    pp, score = generate_midi(file_name, 
                              chord_types, 
                              in_note_array_pitch_corrected[:first_n_notes_to_be_used], 
                              in_note_array[:first_n_notes_to_be_used], 
                              in_scale, in_key, 
                              max_epochs_without_improvement=max_epochs_without_improvement, 
                              epochs=epochs, 
                              population_size=pop_size, 
                              replacement_ratio=repl_ratio, 
                              mutation_ratio=mut_ratio, 
                              combination_ratio=comb_ratio, 
                              new_population=True, 
                              show_output=False
                              )
    
    # writes log-information of the currently produced harmonization
    log_file_pointer.write("\n")
    log_file_pointer.write("---------------------")
    log_file_pointer.write("\n")
    log_file_pointer.write(file_name)
    log_file_pointer.write("\n")
    log_file_pointer.write(f"SCORE: {score}")
    log_file_pointer.write("\n")
    log_file_pointer.write("---------------------")
    log_file_pointer.write("\n")
    
    # checks if a better score has been produced
    if score < best_score:
        best_score = score
        best_file_name = file_name
        
    # some additional whitespaces to separate hyperparameter configurations in the console
    print("\n\n")

# writes log-information of the best produced harmonization during hyperparameter-search
log_file_pointer.write("\n")
log_file_pointer.write("---------- BEST SCORE -----------")
log_file_pointer.write("\n")
log_file_pointer.write(best_file_name)
log_file_pointer.write("\n")
log_file_pointer.write(f"SCORE: {best_score}")
log_file_pointer.write("\n")
log_file_pointer.write("---------- BEST SCORE -----------")
log_file_pointer.write("\n")

log_file_pointer.flush()
log_file_pointer.close()

# prints the best score to the console as well
print("---------- BEST SCORE -----------")
print("\n")
print(best_file_name)
print(f"SCORE: {best_score}")
print("\n")
print("---------- BEST SCORE -----------")

START OF HARMONIZATION PROCESS... 

FirstNotes: 20
PopSize: 25
ReplRatio: 0.1
MutRatio: 0.1
CombRatio: 0.1

Fitness: 637.022745587002

END OF HARMONIZATION PROCESS


C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=3 note=72 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=4 note=79 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=1 note=101 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=1 note=87 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off chan




START OF HARMONIZATION PROCESS... 

FirstNotes: 20
PopSize: 25
ReplRatio: 0.1
MutRatio: 0.2
CombRatio: 0.1

Fitness: 590.0435943541888

END OF HARMONIZATION PROCESS


C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=1 note=89 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=4 note=68 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=1 note=99 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")





START OF HARMONIZATION PROCESS... 

FirstNotes: 20
PopSize: 25
ReplRatio: 0.2
MutRatio: 0.1
CombRatio: 0.1

Fitness: 616.111379876432

END OF HARMONIZATION PROCESS



START OF HARMONIZATION PROCESS... 

FirstNotes: 20
PopSize: 25
ReplRatio: 0.2
MutRatio: 0.2
CombRatio: 0.1


...

Fitness: 585.0202838179724

END OF HARMONIZATION PROCESS



START OF HARMONIZATION PROCESS... 

FirstNotes: 20
PopSize: 50
ReplRatio: 0.1
MutRatio: 0.2
CombRatio: 0.1

Fitness: 572.0118902888574

END OF HARMONIZATION PROCESS


C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=4 note=60 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")





START OF HARMONIZATION PROCESS... 

FirstNotes: 20
PopSize: 50
ReplRatio: 0.2
MutRatio: 0.1
CombRatio: 0.1

Fitness: 570.0003391351703

END OF HARMONIZATION PROCESS



START OF HARMONIZATION PROCESS... 

FirstNotes: 20
PopSize: 50
ReplRatio: 0.2
MutRatio: 0.2
CombRatio: 0.1


C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=4 note=75 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")
C:\Users\chfei\anaconda3\envs\miws23\lib\site-packages\partitura\io\importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=4 note=63 velocity=64 time=0
  warnings.warn(f"ignoring MIDI message {msg}")


Fitness: 567.0106326146168

END OF HARMONIZATION PROCESS



---------- BEST SCORE -----------


./hyperparam/HyperParamSearch_FirstNotes_20_PopSize_50_ReplRatio_0.2_MutRatio_0.2_CombRatio_0.1
SCORE: 567.0106326146168


---------- BEST SCORE -----------
